In [1]:
import numpy as np

In [2]:
def adj_d(a, b, adj): return adj[a, b]


def euc_2d(a, b): return np.linalg.norm(x=a-b, ord=2)


def att(a, b):
    d = euc_2d(a, b)
    d_ = float(int(d))

    if d_ < d:
        return d_ + 1.
    else:
        return d_

In [49]:
def make_genes(idxs, n):
    def make_gene(): return np.random.permutation(idxs)
    return [make_gene().tolist() for i in range(0, n)]


def ord_crossover(a, b, window=5, verbose=False):

    p1 = np.random.randint(0, len(a))

    p2 = (p1 + window) % len(a)

    l = len(a)

    if p2 < p1:
        p1, p2 = p2, p1

    o1 = a[p1:p2]
    o2 = b[p1:p2]

    if verbose:
        print(o1, o2)

    a = [e for e in a[p2:]+a[:p2] if e not in o2]
    b = [e for e in b[p2:]+b[:p2] if e not in o1]

    if verbose:
        print(b[l-p2:]+["o1"]+b[:l-p2], a[l-p2:]+["o2"]+a[:l-p2])

    o1 = b[l-p2:]+o1+b[:l-p2]
    o2 = a[l-p2:]+o2+a[:l-p2]

    return o1, o2


def disp_mutation(a, window=5, p=0.01, verbose=False):
    bit_mask = np.random.choice(a=[True, False], size=len(a), p=[p, 1-p])

    for i, mut in enumerate(bit_mask):
        if mut:
            p1 = i
            p2 = (p1 + window) % len(a)

            if p2 < p1:
                p1, p2 = p2, p1

            if verbose:
                print(a[p1: p2])

            a__ = [e for e in a if e not in a[p1: p2]]
            p3 = np.random.randint(0, len(a__))

            a = a__[:p3]+a[p1: p2]+a__[p3:]

    return a


def fitness(gene, adj, func):
    if func == adj_d:
        return sum([func(gene[i], gene[(i+1)%len(gene)], adj) for i in range(0, len(gene))])
    else:
        return sum([func(adj[gene[i]], adj[gene[(i+1)%len(gene)]]) for i in range(0, len(gene))])

In [60]:
class EvolutionTSP():
    def __init__(self, path, population_n, cross=ord_crossover, mutation=disp_mutation):
        self.cross = cross
        self.mutation = mutation
        self.adj = np.random.random((10,10))
        self.fit_type = adj_d

        self.population = make_genes(len(self.adj), population_n)
        pass
    
    def get_best(self):
        chromo = sorted(self.population, key=lambda x : fitness(x, self.adj, self.fit_type))[0]
        score = fitness(chromo, self.adj, self.fit_type)
        return chromo, score

    def __call__(self):
        self.population = sorted(self.population, key=lambda x : fitness(x, self.adj, self.fit_type))
        l = len(self.population)
        l_4 = int(l/4)

        next_generation = []

        for i in range(0, int(l/2)):
            g1 = np.random.choice(a=[0, 1, 2, 3], p=[0.5, 0.3, 0.15, 0.05])
            g2 = np.random.choice(a=[0, 1, 2, 3], p=[0.5, 0.3, 0.15, 0.05])
            p1 = g1 + np.random.randint(0, l_4)
            p2 = g2 + np.random.randint(0, l_4)
            o1, o2 = self.cross(self.population[p1], self.population[p2])
            next_generation.append(self.mutation(o1))
            next_generation.append(self.mutation(o2))

        self.population = next_generation

In [61]:
tsp = EvolutionTSP(None, 10)

In [62]:
tsp.population

[[2, 8, 5, 9, 1, 6, 4, 7, 0, 3],
 [5, 8, 3, 9, 0, 2, 7, 6, 4, 1],
 [8, 2, 6, 7, 3, 4, 5, 1, 0, 9],
 [3, 0, 8, 5, 1, 2, 9, 6, 7, 4],
 [2, 7, 8, 5, 6, 4, 3, 9, 0, 1],
 [3, 5, 8, 2, 1, 9, 6, 7, 0, 4],
 [5, 0, 2, 4, 1, 7, 3, 8, 6, 9],
 [2, 7, 1, 5, 9, 4, 0, 3, 8, 6],
 [3, 9, 6, 8, 4, 7, 1, 0, 2, 5],
 [9, 3, 7, 4, 0, 2, 1, 6, 8, 5]]

In [63]:
tsp()

In [64]:
tsp.population

[[5, 3, 7, 4, 0, 2, 1, 8, 6, 9],
 [9, 0, 2, 4, 1, 7, 3, 6, 8, 5],
 [2, 7, 8, 5, 6, 4, 3, 9, 0, 1],
 [2, 7, 8, 5, 6, 4, 3, 9, 0, 1],
 [7, 3, 8, 5, 1, 2, 9, 6, 0, 4],
 [5, 9, 2, 4, 1, 7, 3, 6, 0, 8],
 [4, 0, 8, 5, 1, 2, 9, 6, 3, 7],
 [5, 4, 0, 2, 1, 6, 9, 7, 3, 8],
 [9, 4, 0, 2, 1, 7, 3, 8, 6, 5],
 [5, 4, 6, 7, 3, 0, 2, 1, 8, 9]]